In [3]:
using Pkg
# import Pkg; Pkg.add("DifferentialEquations")
# import Pkg; Pkg.add("DiffEqCallbacks")
# import Pkg; Pkg.add("Plots")
# Pkg.add("PGFPlots")
Pkg.activate("..")

using DifferentialEquations, DiffEqCallbacks, Plots, Printf, LaTeXStrings

In [4]:
function dM(dM,M,p,t)
    intak = p[:a_0] * (abs(M[1]) ^ p[:a_b])
    maint = p[:b_0] * (abs(M[1]) ^ p[:b_b])

    if t < p[:Alph] 
        repro = 0.0
    else
        repro = p[:c_0] * (abs(M[1]) ^ p[:c_b])
        
        end
        
    dM[1] = intak - maint - repro
    dM[2] = repro * exp(-(p[:k]+p[:Z])*(t-p[:Alph]))    
end

dM (generic function with 1 method)

In [5]:
hypothetical_starting_mass = 0.1
hypothetical_asymptotic_mass = 25000
hypothetical_starting_reproduction = 0 #c * (hypothetical_starting_mass)^ rho
a = 0.7; a_b = 0.85
b = a/(hypothetical_asymptotic_mass^0.25)
c = 0.001 # West et al. assume 0.1 from Peters
alpha = 0.00001
k_vec = [0.01, 0.1]
a_b_vec = [0.75, 0.85]
resolution = 100
c_vec = range(0.001,0.5,length = resolution)
c_b_vec = range(0.001,1.25,length = resolution)
xticks = 0:10:100
yticks = 0:10:100

0:10:100

In [6]:
for g in 1:length(k_vec)
    k = k_vec[g]
    for h in 1:length(a_b_vec)
        a_b = a_b_vec[h]
        p = Dict([(:a_0,a),(:a_b,a_b),
                  (:b_0,b),(:b_b,1.0),
                  (:c_0,c),(:c_b,1.0),
                  (:Alph,alpha),
                  (:k,k),(:Z,2/alpha)])
        M0 = [hypothetical_starting_mass, hypothetical_starting_reproduction]
        tspan = (0.0,1e4)
        prob = ODEProblem(dM,M0,tspan,p)
        sol = solve(prob,Rosenbrock23())
        M_alph = sol(p[:Alph])[1] # find m_alpha for your given alpha
        
        ## Calculate which curves have an asymptotic size larger than m_alpha (i.e. no shrinking) ##
        results_no_shrink = Array{Float64,2}(undef,resolution,resolution)
        for i in 1:resolution
            c_0 = c_vec[i]
            for j in 1:resolution
                c_b = c_b_vec[j]
                p = Dict([(:a_0,a),(:a_b,a_b),
                  (:b_0,b),(:b_b,1.0),
                  (:c_0,c_0),(:c_b,c_b),
                  (:Alph,0.0),
                  (:k,k),(:Z,0.9)])
                prob = ODEProblem(dM,M0,tspan,p)
                sol = solve(prob,Rosenbrock23())
                results_no_shrink[i,j] = sol[end][1]
            end
        end
        
        feas = results_no_shrink .> M_alph # matrix of feasible growth curves where asmyptotic size is larger than mass a maturity (i.e.) those curves with no shrinking
        ## Of those feasible curves, what is optimum
        results_optimisation = Array{Any,2}(undef,resolution,resolution)
        M0 = [hypothetical_starting_mass, hypothetical_starting_reproduction]
        for i in 1:resolution
            c_0 = c_vec[i]
            for j in 1:resolution
                c_b = c_b_vec[j]
                p = Dict([(:a_0,a),(:a_b,a_b),
                  (:b_0,b),(:b_b,1.0),
                  (:c_0,c_0),(:c_b,c_b),
                  (:Alph,alpha),
                  (:k,k),(:Z,2/alpha)])
                prob = ODEProblem(dM,M0,tspan,p)
                sol = solve(prob,Rosenbrock23())
                results_optimisation[i,j] = sol
            end
        end
        x = [i.u[end][2] for i = results_optimisation] # find max reproduction for each cell
        ## Find optimum values from matrix
        max = findmax(x .* feas)
        c_opt = c_vec[max[2][1]]
        rho_opt = c_b_vec[max[2][2]]
        ## Set plot attrivutes
        xticks_labels = 0.00:maximum(c_vec)/10:maximum(c_vec)
        yticks_labels = 0.00:maximum(c_b_vec)/10:maximum(c_b_vec)
        ## Generate plots
        heatmap(x .* feas, xlab=L"c",xticks = (xticks,xticks_labels),
                            ylab=L"\rho", yticks = (yticks,yticks_labels),
                            transpose=true)
        annotate!([(80, 90, text("c opt = $(@sprintf("%.2f", c_opt))\n  rho opt = $(@sprintf("%.2f", rho_opt))", 10, :white, :topright))])
        annotate!([(max[2][1], max[2][2], text("O", 12, :white))])
        ## Save fitness surface
        file_name = "../Results/opt_hm_Alph=$(@sprintf("%.2f", alpha))_a=$(@sprintf("%.2f", a))_x=$(@sprintf("%.2f", a_b))_k=$(@sprintf("%.2f", p[:k])).pdf"
#         savefig(file_name) # Saves the plot from p as a .pdf vector graphic
        ## Save parameter space for feasible curves
#         heatmap(feas, xlab=L"c",xticks = (xticks,xticks_labels),
#                             ylab=L"\rho", yticks = (yticks,yticks_labels),
#                             transpose=true) #, colormap=ColorMaps.Named("Jet"))
#         savefig("../Results/feasible.pdf") # Saves the plot from p as a .pdf vector graphic
    end
end